EDA 2

focus on word frequency and stopword database

May need to custom build stopword just for LDA ??

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from pathlib import Path
import random

random.seed(13)

dataset_heartless_path = Path('../../dataset/sa/dataset_heartless_20240116.pkl').resolve()

dataset = pd.read_pickle(dataset_heartless_path)
# dataset = dataset.sample(frac=p)      # no sampling is needed

# convert the text to string object
dataset['review_text'] = dataset['review_text'].astype('str')

# drop any duplicate just in case
dataset = dataset.drop_duplicates(keep='first', subset=['review_text', 'review_score', 'review_votes', 'app_id'])

# replace -1 to 0
# then 0 = negative, 1 = positive
# for easier processing
dataset['review_score'] = dataset['review_score'].replace(-1, 0)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4180148 entries, 0 to 4891927
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   index         int64 
 1   app_id        int64 
 2   app_name      object
 3   review_text   object
 4   review_score  int64 
 5   review_votes  int64 
dtypes: int64(4), object(2)
memory usage: 223.2+ MB


Playground to test various pre-processing specailly for topic modelling

In [18]:
# data preprocessing

import re

def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", ' ', raw)      # remove html, or xml tags
    result = re.sub('&gt;', ">", result)
    result = re.sub('&lt;', "<", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', ' ', result)
    result = re.sub('&#62;', ' ', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", ' ', result)
    return result

def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)

def remove_links(x):
    '''Ref: https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python'''
    cleaned_string = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', ' ', x)
    return cleaned_string

def remove_links2(x):
    '''Ref: https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python'''
    cleaned_string = re.sub('[^ ]+\.[^ ]+',' ',x)
    return cleaned_string


def unify_whitespaces(x):
    cleaned_string = re.sub(' +', ' ', x)
    return cleaned_string

from nltk.corpus import stopwords
stop=set(stopwords.words("english"))
def remove_stopword(text):
   text=[word.lower() for word in text.split() if word.lower() not in stop]
   return " ".join(text)

# only keep alphabets
def remove_non_alphabets(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return text

def cleaning(df, review):
    df[review] = df[review].apply(lambda x: remove_links(x))
    df[review] = df[review].apply(lambda x: remove_links2(x))
    df[review] = df[review].apply(lambda x: clean(x))
    df[review] = df[review].apply(lambda x: deEmojify(x))
    df[review] = df[review].apply(lambda x: remove_non_alphabets(x))
    df[review] = df[review].apply(lambda x: x.lower())
    df[review] = df[review].apply(lambda x: unify_whitespaces(x))
    df[review] = df[review].apply(lambda x: remove_stopword(x))

In [19]:
# apply data preprocessing

cleaning(dataset, 'review_text')

In [20]:
# get word count of all reviews

from collections import Counter

def get_word_frequency(df, review):
    word_freq = Counter()
    for sentence in df[review].values:
        word_freq.update(sentence.split())
    return word_freq

word_freq = get_word_frequency(dataset, 'review_text')

# get the number of unique words
print(len(word_freq))

# get the most common words
print(word_freq.most_common(1000))

581651
[('game', 6384611), ('like', 1349853), ('play', 1089248), ('good', 1012459), ('one', 994980), ('get', 972518), ('fun', 966928), ('great', 852133), ('really', 851573), ('games', 836411), ('time', 770477), ('story', 710529), ('would', 665477), ('even', 609491), ('much', 560838), ('played', 520808), ('well', 513021), ('best', 501376), ('first', 476455), ('still', 463988), ('playing', 458463), ('also', 449638), ('gameplay', 434690), ('make', 414268), ('love', 392149), ('hours', 386086), ('buy', 382330), ('way', 373727), ('new', 368916), ('recommend', 364334), ('want', 359334), ('better', 359086), ('worth', 348116), ('lot', 348082), ('people', 341369), ('many', 333334), ('ever', 331922), ('graphics', 328118), ('go', 320540), ('pretty', 319729), ('little', 298737), ('amazing', 290992), ('could', 287548), ('every', 286277), ('feel', 284013), ('say', 283240), ('got', 281064), ('think', 280756), ('bad', 271882), ('bit', 269147), ('see', 263434), ('though', 263414), ('things', 261858), ('

In [21]:
# create a dataframe from word_freq

word_freq_df = pd.DataFrame(word_freq.most_common(len(word_freq)), columns=['word', 'freq'])

word_freq_df.head()

,word,freq
0,game,6384611
1,like,1349853
2,play,1089248
3,good,1012459
4,one,994980


In [23]:
word_freq_df[word_freq_df['word'].str.contains('https')]

,word,freq
63538,https,17
85450,httpsendrequest,10


In [7]:
word_freq_df.iloc[:1000]

,word,freq
0,game,6442590
1,like,1353795
2,play,1093181
3,good,1018245
4,one,999869
...,...,...
995,army,23154
996,throw,23152
997,survive,23129
998,fresh,23066


In [33]:
# save the df to csv

word_freq_df.to_csv('word_freq.csv', index=False)

In [9]:
import spacy
from string import punctuation
from spacy.lang.en import stop_words

import nltk
from nltk.corpus import stopwords

In [12]:
spacy_stopwords = set(spacy.lang.en.stop_words.STOP_WORDS)
nltk_stopwords = set(stopwords.words('english'))

In [14]:
same = spacy_stopwords.intersection(nltk_stopwords)
print(same)

{'doing', 'herself', 'both', 'above', 'between', 'not', 'in', 'all', 'and', 'am', 'further', 'myself', 'while', 'me', 'again', 'nor', 'this', 'each', 'until', 'who', 'where', 'same', 'after', 'from', 'you', 'whom', 'yourselves', 'were', 'her', 'itself', 'through', 'was', 'are', 'him', 'to', 'or', 'which', 'then', 'here', 'other', 'yours', 'his', 'hers', 'been', 'should', 'such', 'very', 'will', 'had', 'my', 'at', 'they', 'when', 'with', 'no', 'but', 'did', 'ours', 'about', 'does', 'himself', 'too', 'of', 'own', 'below', 'an', 'now', 'there', 'that', 'why', 'being', 'these', 'if', 'out', 'on', 'i', 'yourself', 'off', 'most', 'as', 'into', 'a', 'themselves', 'is', 'it', 'their', 'how', 'she', 'we', 'because', 'just', 'your', 'do', 'any', 'before', 'during', 'have', 'for', 'more', 'what', 'the', 'them', 'under', 'only', 'its', 'he', 'over', 'can', 'some', 'by', 'once', 'against', 'our', 'so', 'than', 'ourselves', 'few', 'down', 're', 'has', 'those', 'be', 'up'}


In [17]:
symmetric_diff = spacy_stopwords ^ nltk_stopwords
print(symmetric_diff)

{'made', 'nothing', 'eight', 'hence', 'across', 'eleven', 'whatever', 'whither', 'll', 'm', 'get', 'last', 'wasn', 'ten', 'haven', 'otherwise', 'noone', 's', 'due', "n't", "couldn't", 'us', 'elsewhere', 'must', 'show', 'therein', 'became', 'seeming', 'nobody', 'also', 'thence', "'ll", 'whose', 'somehow', 'four', 'namely', "hasn't", "shan't", 'everything', "needn't", '’re', 'nowhere', 'needn', 've', 'latter', 'although', "'d", 'never', 'theirs', 'go', 'keep', 'often', 'afterwards', 'even', '’m', 'many', 'nevertheless', 'unless', 'hereupon', 'isn', 'take', 'hasn', 'without', 'seem', 'per', "mustn't", 'call', "didn't", 'twelve', 'please', "'m", 'ca', 'thus', 'hereby', 'wherein', 'every', 'various', 'back', 'whereupon', 'amongst', '‘s', "you've", 'give', 'might', 'serious', 'besides', 'together', 'fifteen', 'fifty', "don't", 'become', 'wouldn', 'quite', 'say', 'y', 'none', 'whereas', 'others', 'whoever', 'yet', 'several', 'within', 'moreover', 'amount', 'via', "it's", 'couldn', "hadn't", '

In [18]:
diff = nltk_stopwords - spacy_stopwords
print(diff)

{"should've", "mustn't", 'didn', "didn't", "hasn't", "shan't", 'o', 'weren', "you'll", "needn't", "isn't", "haven't", "mightn't", 'll', 'shouldn', "wasn't", 't', "doesn't", 'm', 'ain', "it's", 'having', "shouldn't", 'couldn', "hadn't", 'ma', 'needn', 've', 'y', 'wasn', "you'd", "aren't", "won't", 'haven', 'mustn', 's', "that'll", "wouldn't", 'theirs', 'hadn', "you've", 'shan', "couldn't", "you're", 'doesn', "weren't", 'don', 'aren', "don't", 'mightn', 'isn', 'wouldn', "she's", 'hasn', 'd', 'won'}


In [19]:
diff2 = spacy_stopwords - nltk_stopwords
print(diff2)

{'made', 'nothing', 'eight', 'hence', 'much', 'five', 'across', 'eleven', 'whatever', 'make', 'another', 'indeed', 'whither', 'onto', 'whether', 'sometimes', 'anything', 'get', 'last', 'ever', 'something', '‘re', 'ten', 'otherwise', 'noone', 'due', 'sixty', "n't", 'becoming', 'us', 'n’t', 'elsewhere', 'must', 'hundred', 'would', 'show', 'therein', 'became', 'front', 'twenty', 'seeming', 'somewhere', 'toward', 'nobody', 'mostly', 'also', 'thence', "'ll", "'s", 'done', 'using', 'thereby', 'whose', 'seemed', 'except', 'alone', 'enough', 'somehow', 'therefore', 'seems', 'four', 'rather', 'namely', '’s', 'forty', 'everything', 'herein', 'whereby', '’re', 'along', 'anywhere', 'nowhere', 'whenever', 'latter', 'although', "'d", 'cannot', 'n‘t', 'never', 'hereafter', 'see', 'third', 'go', 'keep', 'often', 'afterwards', 'even', 'whole', '’m', 'many', 'nevertheless', 'unless', 'hereupon', 'take', 'without', 'since', 'bottom', 'seem', 'sometime', 'top', 'always', 'though', 'per', 'move', 'call', '

Combining more stopwords may achieve better performance in topic modelling

by removing connectives, appreprositions, pronouns, ...

---

TODO: Build custom stopword list for LDA